In [1]:
import gym
import gym_simpleflappy
from tqdm import tqdm
import neat
from matplotlib import pyplot as plt


env = gym.make("SimpleFlappyDistance-v0")
env.reset()

max_eps = 1000
episode = 0
steps = 0
screen_width = 288
screen_height = 512
max_speed = 10.0

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\users\sofik\appdata\local\programs\python\python37\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Environment '<class 'gym_simpleflappy.envs.flappy_env.FlappyEnvDistance'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
observation = env.reset()

In [3]:
observation, reward, done, info = env.step(1)
env.render(mode='human')
print(observation)

[247.          -9.         168.8        234.15441816 320.8
 373.17276174]


In [4]:
observation, reward, done, info = env.step(0)
env.render(mode='human')
print(observation)

[239.          -8.         164.8        234.15441816 316.8
 373.17276174]


In [9]:
def normalize_features(features):
    features[3], features[5] = features[3]/screen_height, features[5]/screen_height
    features[2], features[4] = features[3]/screen_width, features[5]/screen_width
    features[1] /= max_speed
    return features


def eval_genomes(individuals, config):
    """
    runs the game for each of the individuals
    :param config: config file from neat module
    :param individuals: list of individuals from one population
    :return: None
    """
    global env
    env.reset()
    nets, genomes = [], []
    
    for genome_id, genome in tqdm(individuals):
        genome.fitness = 0  # start with fitness level of 0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        nets.append(net)
        genomes.append(genome)
    
    x = 0
    for net in tqdm(nets):  # Cycle for each individual
        observation = env.reset()  # Constructs an instance of the game
        steps = 0
        
        # Extract features and normalize them
        features = normalize_features(list(observation))
        
        output = nets[x].activate(features)  # Get output from neural network
        action = 0 if output[0] <= 0.5 else 1
        
        observation, reward, done, info = env.step(action)
        genomes[x].fitness += reward
        steps += 1
        
        while not done:  #  Game cycle            
            # Extract features and normalize them
            features = normalize_features(list(observation))
            output = nets[x].activate(features)  # Get input from neural network
            action = 0 if output[0] <= 0.5 else 1

            observation, reward, done, info = env.step(action)  #  Make a move
            steps += 1
            if reward == -1:
                genomes[x].fitness -= 10 if steps > 15 else 100
            else:
                genomes[x].fitness += reward
        x += 1
           

In [10]:
def run(config_file, num_generations):
    """
    runs the NEAT algorithm to train a neural network to play game.
    :param config_file: location of config file
    :return: None
    """
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    population = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    population.add_reporter(neat.StdOutReporter(False))
    stats = neat.StatisticsReporter()
    population.add_reporter(stats)
    
    # Add a checkpointer to save progress
    checkpointer = neat.Checkpointer()
    population.add_reporter(checkpointer)

    steps = 0
    
    # Run for maximun of num_populations
    winner = neat.nn.FeedForwardNetwork.create(
        population.run(eval_genomes, num_generations), config)
    observation = env.reset()  # Constructs an instance of the game    
    features = normalize_features(list(observation))
    output = winner.activate(features)  # Get output from neural network
    action = 0 if output[0] <= 0.5 else 1
    observation, reward, done, info = env.step(action)
    
    while not done:  #  Game cycle            
        features = normalize_features(list(observation))
        output = winner.activate(features)  # Get output from neural network
        action = 0 if output[0] <= 0.5 else 1
        observation, reward, done, info = env.step(action)  #  Make a move
        env.render(mode='human')

    return winner
    


In [13]:
if __name__ == '__main__':
    config_path = 'flappy-config.txt'
    winner = run(config_path, 100)
    print(winner)


 ****** Running generation 0 ****** 



100%|███████████████████████████████████████████████████████████████████████████████| 380/380 [00:01<00:00, 330.07it/s]


Population's average fitness: 29.63684 stdev: 6.49856
Best fitness: 36.00000 - size: (21, 140) - species 3 - id 3
Average adjusted fitness: 0.511
Mean genetic distance 4.162, standard deviation 0.285
Population of 760 members in 385 species
Total extinctions: 0
Generation time: 13.090 sec

 ****** Running generation 1 ****** 



100%|███████████████████████████████████████████████████████████████████████████████| 760/760 [00:02<00:00, 352.85it/s]


Population's average fitness: 29.53289 stdev: 6.69320
Best fitness: 74.00000 - size: (21, 140) - species 53 - id 597
Average adjusted fitness: 0.127
Mean genetic distance 4.162, standard deviation 0.285
Population of 770 members in 385 species
Total extinctions: 0
Generation time: 14.136 sec (13.613 average)

 ****** Running generation 2 ****** 



...

Population's average fitness: 33.28877 stdev: 5.28159
Best fitness: 36.00000 - size: (12, 27) - species 3119 - id 27691
Average adjusted fitness: 0.618
Mean genetic distance 4.497, standard deviation 1.158
Population of 395 members in 108 species
Total extinctions: 0
Generation time: 2.040 sec (2.111 average)
Saving checkpoint to neat-checkpoint-97

 ****** Running generation 98 ****** 



100%|███████████████████████████████████████████████████████████████████████████████| 395/395 [00:00<00:00, 770.49it/s]


Population's average fitness: 32.47848 stdev: 5.77743
Best fitness: 36.00000 - size: (20, 45) - species 3121 - id 27963
Average adjusted fitness: 0.638
Mean genetic distance 4.413, standard deviation 1.131
Population of 368 members in 101 species
Total extinctions: 0
Generation time: 2.155 sec (2.067 average)

 ****** Running generation 99 ****** 



100%|███████████████████████████████████████████████████████████████████████████████| 368/368 [00:00<00:00, 725.98it/s]


Population's average fitness: 32.82065 stdev: 5.58778
Best fitness: 36.00000 - size: (16, 38) - species 3155 - id 28252
Average adjusted fitness: 0.651
Mean genetic distance 4.347, standard deviation 1.086
Population of 376 members in 107 species
Total extinctions: 0
Generation time: 2.061 sec (2.072 average)


In [ ]:
# Save winner's progress to file

import pickle
with open('winner-ctrnn', 'wb') as f:
    pickle.dump(winner, f)